# Reading shp file

The shp file contains the polygons with classes and the code below transforms it to bounding boxes

In [0]:
import fiona

In [0]:
shape = fiona.open("../data/2018/PDigital2018_23367__pol.shp")
print(shape.schema)

{'properties': OrderedDict([('linkcolumn', 'str:255'), ('uf', 'str:99'), ('pathrow', 'str:20'), ('mainclass', 'str:255'), ('class_name', 'str:255'), ('dsfnv', 'float:32'), ('julday', 'float:32'), ('view_date', 'str:255'), ('ano', 'float:32'), ('scene_id', 'float:32'), ('areameters', 'float:40.15')]), 'geometry': 'Polygon'}


In [0]:
#first feature of the shapefile
first = shape.next()
print(first) # (GeoJSON format)

{'type': 'Feature', 'id': '110', 'properties': OrderedDict([('linkcolumn', '217622'), ('uf', 'RO'), ('pathrow', '23367'), ('mainclass', 'DESMATAMENTO'), ('class_name', 'd2007'), ('dsfnv', 0.0), ('julday', 0.0), ('view_date', None), ('ano', 2007.0), ('scene_id', 0.0), ('areameters', 98149.4908131361)]), 'geometry': {'type': 'Polygon', 'coordinates': [[(-64.98517996144646, -10.43851630001318), (-64.98410396144656, -10.43851630001325), (-64.9841039614465, -10.439592300012553), (-64.98356596144653, -10.439592300012585), (-64.98356596144647, -10.440668300011897), (-64.98410396144644, -10.440668300011856), (-64.98410396144641, -10.441206300011505), (-64.98464196144633, -10.441206300011475), (-64.98464196144631, -10.441744300011129), (-64.98517996144626, -10.441744300011095), (-64.98517996144619, -10.442820300010396), (-64.98679396144608, -10.442820300010299), (-64.98679396144617, -10.44120630001134), (-64.98625596144622, -10.441206300011372), (-64.98625596144628, -10.440130300012067), (-64.9

/Users/thulliodebortoli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  


In [0]:
first['properties']['mainclass']

'DESMATAMENTO'

In [0]:
for polygon in first['geometry']['coordinates']:
    print(get_bounding_box(polygon))

(-65.77079446144158, -9.495884001897219, -65.83521996143608, -9.404480300637736)
(-65.75626846144517, -9.432456300617744, -65.76070696144495, -9.428959300619956)
(-65.80926146143875, -9.463391300594111, -65.81289296143852, -9.461508300595307)
(-65.7894899614407, -9.448058300605096, -65.81208596143885, -9.434608300614526)
(-65.8153139614331, -9.544763800540853, -65.82096296143328, -9.533062300548243)
(-65.76151396143831, -9.533869300551338, -65.76258996143847, -9.529565300554092)
(-65.81302746143226, -9.56023130053099, -65.81692796143211, -9.556465300533295)


In [0]:
def get_bounding_box(polygon):
    '''
    Function to transform a polygon into a rectangle and get center and top_left coordinate
    '''
    min_lat = min(polygon, key = lambda t: t[1])[1]
    min_lon = min(polygon, key = lambda t: t[0])[0]
    max_lat = max(polygon, key = lambda t: t[1])[1]
    max_lon = max(polygon, key = lambda t: t[0])[0]
    
    center_y = (max_lat + min_lat)/2
    center_x = (max_lon + min_lon)/2
    top_left_y = max_lat
    top_left_x = min_lon
    
    return center_x, center_y, top_left_x, top_left_y

In [0]:
from osgeo import gdal

In [0]:
driver = gdal.GetDriverByName('GTiff')
filename = "../data/2018 2/Landsat8_23367_09032018.tif" #path to raster
dataset = gdal.Open(filename)
band = dataset.GetRasterBand(1)

cols = dataset.RasterXSize
rows = dataset.RasterYSize

geotransform = dataset.GetGeoTransform()

In [0]:
x=-65.77079446144158
y=-9.495884001897219

originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]
xOffset = int((x - originX)/pixelWidth)
yOffset = int((y - originY)/pixelHeight)
print(xOffset,yOffset)

-4941 -33479


In [0]:
xoff, a, b, yoff, d, e = dataset.GetGeoTransform()

def pixel2coord(x, y):
    """Returns global coordinates from pixel x, y coords"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

pixel2coord(5, 5)

(148335.0, -1004535.0)

In [0]:
points_list = [ (-65.77079446144158, -9.495884001897219)]

In [0]:
for point in points_list:
    col = int((point[0] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[1] ) / pixelHeight)

    print (row,col)

-33479 -4861
